In [2]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def firsts(list):
    answer=[]
    for pair in list:
        answer=answer+[pair[0]]
    return answer

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jLeoPowerConstantTerm(m,power,multiplier):
    trun=jLeo(power,m,multiplier)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def last_element(listo):return listo[len(listo)-1]
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}

#produces this on the PARI online evaluator (I have inserted the commas):
br=[
1,
47,
2488,
138799,
7976456,
467232200,
27736348480,
1662803271215,
100442427373480,
6103747246289272,
372725876150863808,
22852464771010647496,
1405886026610765892544,
86741060172969340021952,
5365190340823180439326208,
332577246704242939511725615,
20655377769544663820919905000,
1285027807539621869480480977880,
80066610886753513409821525593280,
4995543732566526565060187887772024,
312067903389730540416319245145039936,
19516459352109724206910675815791735872,
1221787478073080268912138739833447254528,
76558881238278398609546573647116818306504,
4801399849802188285872546222298724299377856,
301358552889212442951924121355286655092791360,
18928524108186605379268259069278244869735006720,
1189719542605042010945455887482239233732751142080,
74824958481405101799295401923145498080031496317440,
4708731584940969251488540213411242070133095720768000,
296483323638911778793802123013217365155428610625064960,
18677571039055424502042574350078071038555962934810664495,
1177200955467256907707767829606512556434525730284672082280,
74229820742983998523807878655148660941364964757170232076440,
4682657672641000613276353688819373189604961982881761635174080,
295516785862704112676947743865736338547152307208873658542187480,
18656838683258040776726836797753969443154060448210951169536087360,
1178287550937265649491805466460363896744099593833261406542090821440,
74441259433548426510664621182339422182178689134172479673100078686720,
4704546876230537649051669928635037299315044055233418643313504347890040,
297410696380227510473584821926459754598587577997951261584830786025989440,
18807176292551896455842616399574167855948518855982280636468413444438841280,
1189632505858785415664268185396568316810012962868095237190924015678644805120,
75269436592700558660145646818728077669744495747378078929068356710829357904960,
4763606735739477078702262301306618196904330454342036172567804617626114845601280,
301550219357655322958904198748139655940272138707157414253528789096123355242370560,
19093491105382437947961430595496009051927469794600124607374594862297809973497425920,
1209229421833128214532165231904398024088456532579184673374765702204525386892709582280,
76599462222171488217469562807555444840329820375936645628428503967599842536403748392640,
4853249476279584943018752544135518205835823652569328104071808597099976302206777672382272,
307557161322636240130429378891312616501367843861094990590342588813640658716687276702796288,
19494049310445477564550759598016449348244789590449694853790210431694353584090244058674665536,
1235828235382992966112000337356874542591602729215314383988292324949451319922660010288507026944,
78359386541981263753587663605771220395173695760309511724942887619218844604408365062990458445312,
4969331730256147172835336645727281480799885420753251882375600240456687485460922105379475213365248,
315192836223790555211163444188645825759965673786927108224693473418739558121668975477385609064230080,
19995098780164839843271975968833674908892528077762766665489021901583285029365260596621852077438476800,
1268636632413268672729488723673519658349225434504337043331170274695193536624267473803997222986457986560,
80503572319005599896519802085748511261953758070993177770252662999751870126880338970620619030455487016960,
5109226023515231795174271806642022238278119546486922652746346463140324125835625762209193286894097317188096,
324306079896787161344357124063601140357997183120543361083780244695916059297903737976316852087325736124084224,
20587951238469144728875413572545165973124146169862711202444824611729329948630917798996527215521428049827442688,
1307155886598940172607280751052393343723376964347599361863473016335357927316478471270092962849033836059872428032,
83003437890819298061159220674050513337514687509034267017635549288525596646068731914307086031798152640655708675631,
5271297646867701243231620056348828996858362139183014747261830107569871776680492045283621280463257824831450361228904,
334803625831811164273307471398151444570380979015726639675077730502474540099263374815508109958714263034979870045590040,
21267301347069209180434546119316800827596753601228462222166318226579132663526299067285136882052633679235269979706997440,
1351085301766957831759285167329838127692235945761375100182242807152550787271816136969058195839695684514251809964546971800,
85842015014782910888737358332857326909867676530325317899274801087388530640486646412863052257542933472850576480889534912320,
5454594181103314395353884318297045062026694474478123129221277721208935931363965378785241897523306877686001373372245607417152,
346632363267943630179319892957146981236491202095299908808662717047804243504180203803669349935923567992160546743779715198734848,
22030211252839896655760044436342222596667157343136792244243996326108725373610974640845764035532176284454550965374859141605876696,
1400264005045227498693320089227035884825680282492704474103652886750979292873550448366299419926933438094105780187281431855082851904,
89010607580108363146057947796447019984406819117601908324673767705214888723143847657440514237648023072092581303730690468541358446272,
5658653431087770147783431973072442739707731119479989465299403213540757386414610090412270459203277370032300491403324236240673058231808,
359768282283136451938233183643880156808956764689158714079901896700007719423392215459057530585195812985027222576505521483746145927094080,
22875473496936829687402292014161718834412927763836095316924893494178776078421448526507427925257601899589958056800447532066174066611688960,
1454634188002381701560543002374509285000151863499255133979293536524595484779236558931041033869616374153934389337102738325693541500885040640,
92506668310411752697700027500541107599054510647853447997767226456825933551220718898889759216482942932408592031625495444638507054727309701120,
5883380647402071727818124358464357357061997088366913641247755161122708769301392216445412497947130792332367814389247083022141893681918673035640,
374209367072642703061850255891790471856520862005092473938011251040764312378697398309875157737634899606364054132972840806767260748645213681887040,
23803199270417251468737963122626707467816366171461192749563400557478169477201165362512851772159448598935076348960799452562007578125691327998310080,
1514217230105491006416286735192399705713778112569360921940868647785308212475854144353099514116567802127049424293324092083010142613089652094239352320,
96332413045374746200370571514086365134691986056054938357641223739177015618938978964850794817403946266783867698429445521201166592631737075434674509760,
6128968041798698464871983060993177701445048464359248049771600533082817745158394358756109989641756654653420473533147391458535031474858708357818557350400,
389970917973786025826826148232023527188983145106526374860563378061488842725970203994300386046448714269543873579140545202535929354547230426525139471434240,
24814546341676335548842726357075976008312393230217108797033854347814848781823641548406242672165084256308341602099263161325277248861481206883551599766753280,
1579097865621621078127966349210038916312383695762720347829578839794195077696910123713050602263102661325782769915208166009373762048848807388370046579798291520,
100493898862353192639599085772479279507605308104393463045761183781449039477596456154787934584841158653853713852389459365917827655606822211804400142188630504960,
6395841086583513721518708585123859851483827511052356442692606484016955654747071476807355907729334535301712240745743084597082858081715137457733426771133436244480,
407082422122816769108928976084224767774083074925265772652509091404070682745260601092117551854989868576150378265801324011663291789547097044490460289150388722831360,
25911536639811395996751656925990326528992266850810482361355730044302538357241007798389242049209348961586470395969302758332164645822911593670231053397678686836037120,
1649413547559894591700394303117536943729317363777978884992676745553087694293098678096611231647446457954863017134436134803435906819926459313995976199288693719229255680,
105000403824773770333649105789820480034335517728840968988334310643726824123735041790377778857595055184234699203779438530862092851011368656739154398110404364696677847040,
6684622341158487811999668275612452018461145981041862416511208589860402818472403829783593014653257049064920715626593324371966426230798803277918448269747031588460412436480,
425585443813049916737778209891287760908166309712249413374261870396364033955750883326966977701620958916811619741413201896787531939944010422188516605008297556440177053390280,
27096933232548903092613115617140470313230857234611666991627504070866204307527763605986822544358886296299818124185350044201435781986714935884426869011696899042312957811757760,
1725347275200146778788065707502165945462707429554800950293313680035690477290460889039398857128345130378453865634762702538349329424603778897699271642610584658725955102258591040,
109864008941653383497254010859226498595190791378492674731068039950918263737791469856053624392850573136737164626778125500188292591996295723267255817207828065983109119673561029120,
6996107097499132971946127571962655485323611430800974100070703369809020527942862074490096967477739069312314066266444140870911945979977377211097174725520015644037734486845682825024]

def broadhurst(n):return br[n-1]

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

print("okay")

okay


In [ ]:
#output snipped; I interrupted this after k = 94.
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for power in [1..100]:
    start=time.time()
    data=[]
    for m in [3..3*power+3]:
        cst= jpowerConstantTerm(m,power)
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=ZZ(numericalfactor(rl))
    difference=nf/24-nszfunction(power)
    tests=tests+[difference]
    finish=time.time()
    print("(power,difference,time): ",(power,difference,finish-start))
print("===============================================================================")
print("tests:")
print(tests)
#below was originally done in a separate cell
#import pickle
#wfile = open('/Users/barrybrent/26oct22no1.txt','wb')
#t = pickle.dumps(str(polydata))
#wfile.write(t)
#wfile.close()
---------------------------------------------------------------------------------------------------
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for power in [95..100]:
    start=time.time()
    data=[]
    for m in [3..3*power+3]:
        cst=QQ(jpowerConstantTerm(m,power))
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=ZZ(numericalfactor(rl))
    difference=nf/24-broadhurst(power)
    tests=tests+[difference]
    finish=time.time()
    print("(power,difference,time): ",(power,difference,finish-start))
print("===============================================================================")
print("tests:")
print(tests)
import pickle
wfile = open('/Users/barrybrent/26oct22no2.txt','wb')
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()
print("tests:")
print(tests)

In [2]:
a=[1,2,3];b=[4,5,6];c=a+b;print(c)

[1, 2, 3, 4, 5, 6]


In [6]:
import pickle
rsfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rsfile)
rsfile.close()
s=stripQuotationMarks(rs)

rtfile = open('/Users/barrybrent/26oct22no2.txt','rb')
rt=pickle.load(rtfile)
rtfile.close()
t=stripQuotationMarks(rt)

u=s+t

wufile = open('/Users/barrybrent/5dec22no1.txt','wb')
u = pickle.dumps(str(u))
wufile.write(u)

1968604

In [8]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print(len(s))

100


In [95]:
# renumbered as conjecture 7

import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
mp=0
data=[]
p=2
tests=[]
tupls=[]
dk=1
for h in [0..len(s)-1]:
        k=s[h][0]
        if digitsum(p,k)==dk: # = 1, so k = 2^b & b = ord_2(k) for some b.
            poly=D(s[h][1])
            for m in [3..4100]:
                if ZZ(mod(m,p))==mp: # mp = 0 => m is even
                    #dm=ZZ(m/p).digits(p)
                    dm=digitsum(p,m)
                    a=order(p,m)
                    mp2=mod(m,p^2) # p = 2, m even =>  mp2 = 0 or 2.
                    if mp2==2: # => m  = 2 (mod 4) & a = ord_2(m) = 1.
                            polym=evaluate(poly,m)
                            o=order(p,polym)
                            test=o-((a+6)*k+1)
                            tupl=(k,m,test)
                            tests=tests+[test]
                            print(tupl)
                            tupls=tupls+[tupl]
ut=uniteList(tests);print();print("tests:",ut)

(1, 6, 0)
(1, 10, 0)
(1, 14, 0)
(1, 18, 0)
(1, 22, 0)
(1, 26, 0)
(1, 30, 0)
(1, 34, 0)
(1, 38, 0)
(1, 42, 0)
(1, 46, 0)
(1, 50, 0)
(1, 54, 0)
(1, 58, 0)
(1, 62, 0)
(1, 66, 0)
(1, 70, 0)
(1, 74, 0)
(1, 78, 0)
(1, 82, 0)
(1, 86, 0)
(1, 90, 0)
(1, 94, 0)
(1, 98, 0)
(1, 102, 0)
(1, 106, 0)
(1, 110, 0)
(1, 114, 0)
(1, 118, 0)
(1, 122, 0)
(1, 126, 0)
(1, 130, 0)
(1, 134, 0)
(1, 138, 0)
(1, 142, 0)
(1, 146, 0)
(1, 150, 0)
(1, 154, 0)
(1, 158, 0)
(1, 162, 0)
(1, 166, 0)
(1, 170, 0)
(1, 174, 0)
(1, 178, 0)
(1, 182, 0)
(1, 186, 0)
(1, 190, 0)
(1, 194, 0)
(1, 198, 0)
(1, 202, 0)
(1, 206, 0)
(1, 210, 0)
(1, 214, 0)
(1, 218, 0)
(1, 222, 0)
(1, 226, 0)
(1, 230, 0)
(1, 234, 0)
(1, 238, 0)
(1, 242, 0)
(1, 246, 0)
(1, 250, 0)
(1, 254, 0)
(1, 258, 0)
(1, 262, 0)
(1, 266, 0)
(1, 270, 0)
(1, 274, 0)
(1, 278, 0)
(1, 282, 0)
(1, 286, 0)
(1, 290, 0)
(1, 294, 0)
(1, 298, 0)
(1, 302, 0)
(1, 306, 0)
(1, 310, 0)
(1, 314, 0)
(1, 318, 0)
(1, 322, 0)
(1, 326, 0)
(1, 330, 0)
(1, 334, 0)
(1, 338, 0)
(1, 342, 0)
(1, 3

(1, 3530, 0)
(1, 3534, 0)
(1, 3538, 0)
(1, 3542, 0)
(1, 3546, 0)
(1, 3550, 0)
(1, 3554, 0)
(1, 3558, 0)
(1, 3562, 0)
(1, 3566, 0)
(1, 3570, 0)
(1, 3574, 0)
(1, 3578, 0)
(1, 3582, 0)
(1, 3586, 0)
(1, 3590, 0)
(1, 3594, 0)
(1, 3598, 0)
(1, 3602, 0)
(1, 3606, 0)
(1, 3610, 0)
(1, 3614, 0)
(1, 3618, 0)
(1, 3622, 0)
(1, 3626, 0)
(1, 3630, 0)
(1, 3634, 0)
(1, 3638, 0)
(1, 3642, 0)
(1, 3646, 0)
(1, 3650, 0)
(1, 3654, 0)
(1, 3658, 0)
(1, 3662, 0)
(1, 3666, 0)
(1, 3670, 0)
(1, 3674, 0)
(1, 3678, 0)
(1, 3682, 0)
(1, 3686, 0)
(1, 3690, 0)
(1, 3694, 0)
(1, 3698, 0)
(1, 3702, 0)
(1, 3706, 0)
(1, 3710, 0)
(1, 3714, 0)
(1, 3718, 0)
(1, 3722, 0)
(1, 3726, 0)
(1, 3730, 0)
(1, 3734, 0)
(1, 3738, 0)
(1, 3742, 0)
(1, 3746, 0)
(1, 3750, 0)
(1, 3754, 0)
(1, 3758, 0)
(1, 3762, 0)
(1, 3766, 0)
(1, 3770, 0)
(1, 3774, 0)
(1, 3778, 0)
(1, 3782, 0)
(1, 3786, 0)
(1, 3790, 0)
(1, 3794, 0)
(1, 3798, 0)
(1, 3802, 0)
(1, 3806, 0)
(1, 3810, 0)
(1, 3814, 0)
(1, 3818, 0)
(1, 3822, 0)
(1, 3826, 0)
(1, 3830, 0)
(1, 3834, 0)

(2, 3722, 0)
(2, 3726, 0)
(2, 3730, 0)
(2, 3734, 0)
(2, 3738, 0)
(2, 3742, 0)
(2, 3746, 0)
(2, 3750, 0)
(2, 3754, 0)
(2, 3758, 0)
(2, 3762, 0)
(2, 3766, 0)
(2, 3770, 0)
(2, 3774, 0)
(2, 3778, 0)
(2, 3782, 0)
(2, 3786, 0)
(2, 3790, 0)
(2, 3794, 0)
(2, 3798, 0)
(2, 3802, 0)
(2, 3806, 0)
(2, 3810, 0)
(2, 3814, 0)
(2, 3818, 0)
(2, 3822, 0)
(2, 3826, 0)
(2, 3830, 0)
(2, 3834, 0)
(2, 3838, 0)
(2, 3842, 0)
(2, 3846, 0)
(2, 3850, 0)
(2, 3854, 0)
(2, 3858, 0)
(2, 3862, 0)
(2, 3866, 0)
(2, 3870, 0)
(2, 3874, 0)
(2, 3878, 0)
(2, 3882, 0)
(2, 3886, 0)
(2, 3890, 0)
(2, 3894, 0)
(2, 3898, 0)
(2, 3902, 0)
(2, 3906, 0)
(2, 3910, 0)
(2, 3914, 0)
(2, 3918, 0)
(2, 3922, 0)
(2, 3926, 0)
(2, 3930, 0)
(2, 3934, 0)
(2, 3938, 0)
(2, 3942, 0)
(2, 3946, 0)
(2, 3950, 0)
(2, 3954, 0)
(2, 3958, 0)
(2, 3962, 0)
(2, 3966, 0)
(2, 3970, 0)
(2, 3974, 0)
(2, 3978, 0)
(2, 3982, 0)
(2, 3986, 0)
(2, 3990, 0)
(2, 3994, 0)
(2, 3998, 0)
(2, 4002, 0)
(2, 4006, 0)
(2, 4010, 0)
(2, 4014, 0)
(2, 4018, 0)
(2, 4022, 0)
(2, 4026, 0)

(4, 2846, 0)
(4, 2850, 0)
(4, 2854, 0)
(4, 2858, 0)
(4, 2862, 0)
(4, 2866, 0)
(4, 2870, 0)
(4, 2874, 0)
(4, 2878, 0)
(4, 2882, 0)
(4, 2886, 0)
(4, 2890, 0)
(4, 2894, 0)
(4, 2898, 0)
(4, 2902, 0)
(4, 2906, 0)
(4, 2910, 0)
(4, 2914, 0)
(4, 2918, 0)
(4, 2922, 0)
(4, 2926, 0)
(4, 2930, 0)
(4, 2934, 0)
(4, 2938, 0)
(4, 2942, 0)
(4, 2946, 0)
(4, 2950, 0)
(4, 2954, 0)
(4, 2958, 0)
(4, 2962, 0)
(4, 2966, 0)
(4, 2970, 0)
(4, 2974, 0)
(4, 2978, 0)
(4, 2982, 0)
(4, 2986, 0)
(4, 2990, 0)
(4, 2994, 0)
(4, 2998, 0)
(4, 3002, 0)
(4, 3006, 0)
(4, 3010, 0)
(4, 3014, 0)
(4, 3018, 0)
(4, 3022, 0)
(4, 3026, 0)
(4, 3030, 0)
(4, 3034, 0)
(4, 3038, 0)
(4, 3042, 0)
(4, 3046, 0)
(4, 3050, 0)
(4, 3054, 0)
(4, 3058, 0)
(4, 3062, 0)
(4, 3066, 0)
(4, 3070, 0)
(4, 3074, 0)
(4, 3078, 0)
(4, 3082, 0)
(4, 3086, 0)
(4, 3090, 0)
(4, 3094, 0)
(4, 3098, 0)
(4, 3102, 0)
(4, 3106, 0)
(4, 3110, 0)
(4, 3114, 0)
(4, 3118, 0)
(4, 3122, 0)
(4, 3126, 0)
(4, 3130, 0)
(4, 3134, 0)
(4, 3138, 0)
(4, 3142, 0)
(4, 3146, 0)
(4, 3150, 0)

(8, 2286, 0)
(8, 2290, 0)
(8, 2294, 0)
(8, 2298, 0)
(8, 2302, 0)
(8, 2306, 0)
(8, 2310, 0)
(8, 2314, 0)
(8, 2318, 0)
(8, 2322, 0)
(8, 2326, 0)
(8, 2330, 0)
(8, 2334, 0)
(8, 2338, 0)
(8, 2342, 0)
(8, 2346, 0)
(8, 2350, 0)
(8, 2354, 0)
(8, 2358, 0)
(8, 2362, 0)
(8, 2366, 0)
(8, 2370, 0)
(8, 2374, 0)
(8, 2378, 0)
(8, 2382, 0)
(8, 2386, 0)
(8, 2390, 0)
(8, 2394, 0)
(8, 2398, 0)
(8, 2402, 0)
(8, 2406, 0)
(8, 2410, 0)
(8, 2414, 0)
(8, 2418, 0)
(8, 2422, 0)
(8, 2426, 0)
(8, 2430, 0)
(8, 2434, 0)
(8, 2438, 0)
(8, 2442, 0)
(8, 2446, 0)
(8, 2450, 0)
(8, 2454, 0)
(8, 2458, 0)
(8, 2462, 0)
(8, 2466, 0)
(8, 2470, 0)
(8, 2474, 0)
(8, 2478, 0)
(8, 2482, 0)
(8, 2486, 0)
(8, 2490, 0)
(8, 2494, 0)
(8, 2498, 0)
(8, 2502, 0)
(8, 2506, 0)
(8, 2510, 0)
(8, 2514, 0)
(8, 2518, 0)
(8, 2522, 0)
(8, 2526, 0)
(8, 2530, 0)
(8, 2534, 0)
(8, 2538, 0)
(8, 2542, 0)
(8, 2546, 0)
(8, 2550, 0)
(8, 2554, 0)
(8, 2558, 0)
(8, 2562, 0)
(8, 2566, 0)
(8, 2570, 0)
(8, 2574, 0)
(8, 2578, 0)
(8, 2582, 0)
(8, 2586, 0)
(8, 2590, 0)

(16, 890, 0)
(16, 894, 0)
(16, 898, 0)
(16, 902, 0)
(16, 906, 0)
(16, 910, 0)
(16, 914, 0)
(16, 918, 0)
(16, 922, 0)
(16, 926, 0)
(16, 930, 0)
(16, 934, 0)
(16, 938, 0)
(16, 942, 0)
(16, 946, 0)
(16, 950, 0)
(16, 954, 0)
(16, 958, 0)
(16, 962, 0)
(16, 966, 0)
(16, 970, 0)
(16, 974, 0)
(16, 978, 0)
(16, 982, 0)
(16, 986, 0)
(16, 990, 0)
(16, 994, 0)
(16, 998, 0)
(16, 1002, 0)
(16, 1006, 0)
(16, 1010, 0)
(16, 1014, 0)
(16, 1018, 0)
(16, 1022, 0)
(16, 1026, 0)
(16, 1030, 0)
(16, 1034, 0)
(16, 1038, 0)
(16, 1042, 0)
(16, 1046, 0)
(16, 1050, 0)
(16, 1054, 0)
(16, 1058, 0)
(16, 1062, 0)
(16, 1066, 0)
(16, 1070, 0)
(16, 1074, 0)
(16, 1078, 0)
(16, 1082, 0)
(16, 1086, 0)
(16, 1090, 0)
(16, 1094, 0)
(16, 1098, 0)
(16, 1102, 0)
(16, 1106, 0)
(16, 1110, 0)
(16, 1114, 0)
(16, 1118, 0)
(16, 1122, 0)
(16, 1126, 0)
(16, 1130, 0)
(16, 1134, 0)
(16, 1138, 0)
(16, 1142, 0)
(16, 1146, 0)
(16, 1150, 0)
(16, 1154, 0)
(16, 1158, 0)
(16, 1162, 0)
(16, 1166, 0)
(16, 1170, 0)
(16, 1174, 0)
(16, 1178, 0)
(16, 1

(16, 3294, 0)
(16, 3298, 0)
(16, 3302, 0)
(16, 3306, 0)
(16, 3310, 0)
(16, 3314, 0)
(16, 3318, 0)
(16, 3322, 0)
(16, 3326, 0)
(16, 3330, 0)
(16, 3334, 0)
(16, 3338, 0)
(16, 3342, 0)
(16, 3346, 0)
(16, 3350, 0)
(16, 3354, 0)
(16, 3358, 0)
(16, 3362, 0)
(16, 3366, 0)
(16, 3370, 0)
(16, 3374, 0)
(16, 3378, 0)
(16, 3382, 0)
(16, 3386, 0)
(16, 3390, 0)
(16, 3394, 0)
(16, 3398, 0)
(16, 3402, 0)
(16, 3406, 0)
(16, 3410, 0)
(16, 3414, 0)
(16, 3418, 0)
(16, 3422, 0)
(16, 3426, 0)
(16, 3430, 0)
(16, 3434, 0)
(16, 3438, 0)
(16, 3442, 0)
(16, 3446, 0)
(16, 3450, 0)
(16, 3454, 0)
(16, 3458, 0)
(16, 3462, 0)
(16, 3466, 0)
(16, 3470, 0)
(16, 3474, 0)
(16, 3478, 0)
(16, 3482, 0)
(16, 3486, 0)
(16, 3490, 0)
(16, 3494, 0)
(16, 3498, 0)
(16, 3502, 0)
(16, 3506, 0)
(16, 3510, 0)
(16, 3514, 0)
(16, 3518, 0)
(16, 3522, 0)
(16, 3526, 0)
(16, 3530, 0)
(16, 3534, 0)
(16, 3538, 0)
(16, 3542, 0)
(16, 3546, 0)
(16, 3550, 0)
(16, 3554, 0)
(16, 3558, 0)
(16, 3562, 0)
(16, 3566, 0)
(16, 3570, 0)
(16, 3574, 0)
(16, 3

(32, 1858, 0)
(32, 1862, 0)
(32, 1866, 0)
(32, 1870, 0)
(32, 1874, 0)
(32, 1878, 0)
(32, 1882, 0)
(32, 1886, 0)
(32, 1890, 0)
(32, 1894, 0)
(32, 1898, 0)
(32, 1902, 0)
(32, 1906, 0)
(32, 1910, 0)
(32, 1914, 0)
(32, 1918, 0)
(32, 1922, 0)
(32, 1926, 0)
(32, 1930, 0)
(32, 1934, 0)
(32, 1938, 0)
(32, 1942, 0)
(32, 1946, 0)
(32, 1950, 0)
(32, 1954, 0)
(32, 1958, 0)
(32, 1962, 0)
(32, 1966, 0)
(32, 1970, 0)
(32, 1974, 0)
(32, 1978, 0)
(32, 1982, 0)
(32, 1986, 0)
(32, 1990, 0)
(32, 1994, 0)
(32, 1998, 0)
(32, 2002, 0)
(32, 2006, 0)
(32, 2010, 0)
(32, 2014, 0)
(32, 2018, 0)
(32, 2022, 0)
(32, 2026, 0)
(32, 2030, 0)
(32, 2034, 0)
(32, 2038, 0)
(32, 2042, 0)
(32, 2046, 0)
(32, 2050, 0)
(32, 2054, 0)
(32, 2058, 0)
(32, 2062, 0)
(32, 2066, 0)
(32, 2070, 0)
(32, 2074, 0)
(32, 2078, 0)
(32, 2082, 0)
(32, 2086, 0)
(32, 2090, 0)
(32, 2094, 0)
(32, 2098, 0)
(32, 2102, 0)
(32, 2106, 0)
(32, 2110, 0)
(32, 2114, 0)
(32, 2118, 0)
(32, 2122, 0)
(32, 2126, 0)
(32, 2130, 0)
(32, 2134, 0)
(32, 2138, 0)
(32, 2

(64, 206, 0)
(64, 210, 0)
(64, 214, 0)
(64, 218, 0)
(64, 222, 0)
(64, 226, 0)
(64, 230, 0)
(64, 234, 0)
(64, 238, 0)
(64, 242, 0)
(64, 246, 0)
(64, 250, 0)
(64, 254, 0)
(64, 258, 0)
(64, 262, 0)
(64, 266, 0)
(64, 270, 0)
(64, 274, 0)
(64, 278, 0)
(64, 282, 0)
(64, 286, 0)
(64, 290, 0)
(64, 294, 0)
(64, 298, 0)
(64, 302, 0)
(64, 306, 0)
(64, 310, 0)
(64, 314, 0)
(64, 318, 0)
(64, 322, 0)
(64, 326, 0)
(64, 330, 0)
(64, 334, 0)
(64, 338, 0)
(64, 342, 0)
(64, 346, 0)
(64, 350, 0)
(64, 354, 0)
(64, 358, 0)
(64, 362, 0)
(64, 366, 0)
(64, 370, 0)
(64, 374, 0)
(64, 378, 0)
(64, 382, 0)
(64, 386, 0)
(64, 390, 0)
(64, 394, 0)
(64, 398, 0)
(64, 402, 0)
(64, 406, 0)
(64, 410, 0)
(64, 414, 0)
(64, 418, 0)
(64, 422, 0)
(64, 426, 0)
(64, 430, 0)
(64, 434, 0)
(64, 438, 0)
(64, 442, 0)
(64, 446, 0)
(64, 450, 0)
(64, 454, 0)
(64, 458, 0)
(64, 462, 0)
(64, 466, 0)
(64, 470, 0)
(64, 474, 0)
(64, 478, 0)
(64, 482, 0)
(64, 486, 0)
(64, 490, 0)
(64, 494, 0)
(64, 498, 0)
(64, 502, 0)
(64, 506, 0)
(64, 510, 0)

(64, 2638, 0)
(64, 2642, 0)
(64, 2646, 0)
(64, 2650, 0)
(64, 2654, 0)
(64, 2658, 0)
(64, 2662, 0)
(64, 2666, 0)
(64, 2670, 0)
(64, 2674, 0)
(64, 2678, 0)
(64, 2682, 0)
(64, 2686, 0)
(64, 2690, 0)
(64, 2694, 0)
(64, 2698, 0)
(64, 2702, 0)
(64, 2706, 0)
(64, 2710, 0)
(64, 2714, 0)
(64, 2718, 0)
(64, 2722, 0)
(64, 2726, 0)
(64, 2730, 0)
(64, 2734, 0)
(64, 2738, 0)
(64, 2742, 0)
(64, 2746, 0)
(64, 2750, 0)
(64, 2754, 0)
(64, 2758, 0)
(64, 2762, 0)
(64, 2766, 0)
(64, 2770, 0)
(64, 2774, 0)
(64, 2778, 0)
(64, 2782, 0)
(64, 2786, 0)
(64, 2790, 0)
(64, 2794, 0)
(64, 2798, 0)
(64, 2802, 0)
(64, 2806, 0)
(64, 2810, 0)
(64, 2814, 0)
(64, 2818, 0)
(64, 2822, 0)
(64, 2826, 0)
(64, 2830, 0)
(64, 2834, 0)
(64, 2838, 0)
(64, 2842, 0)
(64, 2846, 0)
(64, 2850, 0)
(64, 2854, 0)
(64, 2858, 0)
(64, 2862, 0)
(64, 2866, 0)
(64, 2870, 0)
(64, 2874, 0)
(64, 2878, 0)
(64, 2882, 0)
(64, 2886, 0)
(64, 2890, 0)
(64, 2894, 0)
(64, 2898, 0)
(64, 2902, 0)
(64, 2906, 0)
(64, 2910, 0)
(64, 2914, 0)
(64, 2918, 0)
(64, 2